In [6]:
# Load modules necessary
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
import pandas as pd
from scipy.signal import detrend

# %matplotlib qt

In [7]:
# Path to the EEG file
eegPath = '../../Datasets/iEEG/sub-01/eeg/sub-01_task-daf_eeg_filtered.vhdr'

# Load the EEG file using MNE
# MNE has different read formats for different EEG file types
# Here we are using read_raw_edf to read the EEG file
# preload=True loads the data into memory (default is False, which loads the data when needed)
raw = mne.io.read_raw_brainvision(eegPath, preload=True)
elecPos = pd.read_csv('../../Datasets/iEEG/sub-01/eeg/sub-01_electrodes.tsv', sep='\t')
# Add fiducials
fiducials = pd.DataFrame({A
    'name': ['Nz', 'LPA', 'RPA'],
    'x': [-4.129838157917329e-18, -0.0729282673627754, 0.08278152042487033],
    'y': [0.10011015398430487, 3.008505424862354e-18, -3.414981080487009e-18],
    'z': [-5.7777898331617076e-33, 3.851859888774472e-34, 3.4666738998970245e-33]
})

# Concatenate the original electrode positions with the fiducials
elecPos = pd.concat([elecPos, fiducials], ignore_index=True)

montage = mne.channels.make_dig_montage(
    ch_pos=dict(zip(elecPos['name'], elecPos[['x', 'y', 'z']].values)),
    coord_frame='head'
)
raw.set_montage(montage)

Extracting parameters from ../../Datasets/iEEG/sub-01/eeg/sub-01_task-daf_eeg_filtered.vhdr...
Setting channel info structure...
Reading 0 ... 244237  =      0.000 ...   976.948 secs...


<RawBrainVision | sub-01_task-daf_eeg_filtered.eeg, 66 x 244238 (976.9 s), ~123.1 MB, data loaded>

In [8]:
# Let us try plotting an EEG channel using matplotlib
# We can get the data of the EEG channel using get_data() function
# The data is in the form of a numpy array
data = raw.get_data()
chToPlot = 0 # Channel to plot
timeToPlot = 10 # Time in seconds to plot
plt.plot(raw.times[raw.times < timeToPlot], data[chToPlot, raw.times < timeToPlot])
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (uV)')
plt.title('EEG Channel 1')
plt.show()

In [25]:
# Define the parameters for the Morlet wavelet
f0 = 15  # Central frequency of the wavelet
SR = 250  # Sampling rate of the EEG data
sigma = 1  # Standard deviation of the Gaussian window
t = np.arange(-5, 5, 1/SR)  # Time vector
N_ml = len(t)

# Create the Morlet wavelet (frequency based on time)
A = 1 / (np.sqrt(sigma * np.sqrt(np.pi)) * np.sqrt(f0))  # amplitude scaling for the frequency-specific Morlet wavelet
morlet_wavelet = A * np.exp(2 * 1j * np.pi * f0 * t) * np.exp(-t**2 / (2 * sigma**2))

In [10]:
plt.figure()
plt.subplot(3, 1, 1)
plt.title('Cosine (real part of complex exponential)')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.plot(t, np.real(np.exp(2 * 1j * np.pi * f0 * t)))
plt.xlim([t[0], t[-1]])

plt.subplot(3, 1, 2)
plt.title('Gaussian')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.plot(t, np.exp(-t**2 / (2 * sigma**2)))
plt.xlim([t[0], t[-1]])

plt.subplot(3, 1, 3)
plt.title('Wavelet (real part)')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.plot(t, np.real(morlet_wavelet))
plt.xlim([t[0], t[-1]])
plt.show()

In [11]:
signal = np.concatenate([np.zeros(512), np.ones(512), np.zeros(512)])

# Plot the Morlet wavelet and the transformed signal
plt.figure()
plt.subplot(3, 1, 1)
plt.plot(signal, 'r')
plt.title('Original Signal')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.ylim([-1, 2])
plt.xlim([1, len(signal)])

# Apply the Morlet wavelet to the signal (convolution)
wavelet_transformed_signal = np.convolve(signal, morlet_wavelet, 'same')

plt.subplot(3, 1, 2)
plt.plot(t, np.real(morlet_wavelet), 'r', label='Real Part')
plt.plot(t, np.imag(morlet_wavelet), 'b', label='Imaginary Part')
plt.title('Morlet Wavelet')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(np.arange(1, len(signal) + 1), np.real(wavelet_transformed_signal), 'r', label='Real Part')
plt.plot(np.arange(1, len(signal) + 1), np.imag(wavelet_transformed_signal), 'b', label='Imaginary Part')
plt.title('Wavelet Transformed Signal')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

In [26]:
eeg_signal = data[chToPlot, raw.times < timeToPlot]
transformed_signal = np.convolve(eeg_signal, morlet_wavelet, 'same')

plt.figure()
plt.subplot(3, 1, 1)
plt.plot(eeg_signal, 'r')
plt.title('Original EEG Signal')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')

plt.subplot(3, 1, 2)
plt.plot(t, np.real(morlet_wavelet), 'r', label='Real Part')
plt.plot(t, np.imag(morlet_wavelet), 'b', label='Imaginary Part')
plt.title('Morlet Wavelet')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(np.arange(1, len(eeg_signal) + 1), np.real(transformed_signal), 'r', label='Real Part')
plt.plot(np.arange(1, len(eeg_signal) + 1), np.imag(transformed_signal), 'b', label='Imaginary Part')
plt.title('Wavelet Transformed EEG Signal')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.legend()

plt.show()




In [27]:
eeg_signal = data[chToPlot, raw.times < timeToPlot]
eeg_signal[500] = 10000
transformed_signal = np.convolve(eeg_signal, morlet_wavelet, 'same')

plt.figure()
plt.subplot(3, 1, 1)
plt.plot(eeg_signal, 'r')
plt.title('Original EEG Signal')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')

plt.subplot(3, 1, 2)
plt.plot(t, np.real(morlet_wavelet), 'r', label='Real Part')
plt.plot(t, np.imag(morlet_wavelet), 'b', label='Imaginary Part')
plt.title('Morlet Wavelet')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(np.arange(1, len(eeg_signal) + 1), np.real(transformed_signal), 'r', label='Real Part')
plt.plot(np.arange(1, len(eeg_signal) + 1), np.imag(transformed_signal), 'b', label='Imaginary Part')
plt.title('Wavelet Transformed EEG Signal')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.legend()

plt.show()


